In [37]:
import requests
from requests_oauthlib import OAuth1
import cnfg
import pandas as pd
import datetime
import pickle
import time
import json
from pymongo import MongoClient

In [5]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

### Tweets by screen name: Create recursive function to get all tweets based on max_id.

In [57]:
# define screen names for candidates
candidates = {'clinton' : 'hillaryclinton', 
              'trump' : 'realdonaldtrump', 
              'rubio' : 'marcorubio', 
              'sanders' : 'berniesanders', 
              'cruz' : 'tedcruz'}

# define mongodb parameters
coll_dict = {'trump': db.trump, 
             'clinton' : db.clinton, 
             'sanders' : db.sanders, 
             'cruz' : db.cruz, 
             'rubio' : db.rubio}

In [65]:
client = MongoClient()
db = client.tweets

def into_mongo(collection, results):
    coll = collection
    for tweet in results:
        coll.insert_one(tweet)
        
def out_of_mongo(collection):
    tweet_list = []
    for tweet in collection.find():
        tweet_list.append(tweet)
    return tweet_list

In [7]:
max_calls = 150
max_tweets = 3500

In [20]:
### can use this if possible to get > 3200 tweets
# def get_tweets(screen_name, call = 1, max_id = None, tweets = []):
#     if len(tweets) == max_tweets:
#         return tweets 
#     if call > 180: 
#         print 'sleeping'
#         time.sleep(60*15)
#         call = 1 
#     try:
#         print call
#         search_params = {'screen_name': [screen_name], 
#                          'count': 200, 
#                          'max_id': max_id}
#         json = requests.get('https://api.twitter.com/1.1/statuses/user_timeline.json',
#                         params = search_params,
#                         auth = oauth).json()
#         last_id = json[-1]['id']
#         return tweets + get_tweets(screen_name, call + 1, last_id, json)
#     except:
#         return tweets

def get_tweets(screen_name, since_id, call = 1, max_id = None, tweets = []):
    if call > max_calls:
        return tweets
    print call
    search_params = {'screen_name': [screen_name], 
                     'count': 200, 
                     'max_id': max_id, 
                     'since_id' : since_id}
    json = requests.get('https://api.twitter.com/1.1/statuses/user_timeline.json',
                    params = search_params,
                    auth = oauth).json()
    last_id = json[-1]['id']
    try: # catches the first instance 
        if tweets[-1]['id'] == last_id: # check if we ran out of tweets 
            return tweets
        else:
            return tweets + get_tweets(screen_name, since_id, call + 1, last_id, json)
    except:
        return tweets + get_tweets(screen_name, since_id, call + 1, last_id, json)

In [ ]:
# get tweets and insert into mongodb 
for cand in candidates:
    results = get_tweets(candidates[cand], since_id = None)
    into_mongo(coll_dict[cand], results)

In [194]:
## not needed anymore with mongo
# cruz_tweets = get_tweets(candidates['cruz'], since_id = None)
# rubio_tweets = get_tweets(candidates['rubio'], since_id = None)
# hillary_tweets = get_tweets(candidates['clinton'], since_id = None)
# donald_tweets = get_tweets(candidates['trump'], since_id = None)
# bernie_tweets = get_tweets(candidates['sanders'], since_id = None)
# kanye_tweets = get_tweets('kanyewest', since_id = None)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


#### Put old stuff into mongo - delete later?

In [58]:
# with open('tweets/donald_tweets.pkl', 'r') as picklefile:
#     dt_data = pickle.load(picklefile)

# with open('tweets/hillary_tweets.pkl', 'r') as picklefile:
#     hc_data = pickle.load(picklefile)
    
# with open('tweets/bernie_tweets.pkl', 'r') as picklefile:
#     bs_data = pickle.load(picklefile)
    
# with open('tweets/cruz_tweets.pkl', 'r') as picklefile:
#     tc_data = pickle.load(picklefile)

# with open('tweets/rubio_tweets.pkl', 'r') as picklefile:
#     mr_data = pickle.load(picklefile)
    
# all_data = {'trump': dt_data, 'clinton': hc_data, 'sanders': bs_data, 'cruz': tc_data, 'rubio': mr_data}

# for data in all_data:
#     into_mongo(coll_dict[data], all_data[data])
    
# dt_mongo_test = out_of_mongo(db.trump)

**Pickle**

In [23]:
# def pickle_file(tweets):
#     with open('data/%s.pkl' %tweets, 'w') as picklefile:
#         pickle.dump(tweets, picklefile)

In [188]:
# with open('tweets/donald_tweets.pkl', 'w') as picklefile:
#     pickle.dump(donald_tweets, picklefile)
    
# with open('tweets/hillary_tweets.pkl', 'w') as picklefile:
#     pickle.dump(hillary_tweets, picklefile)

# with open('tweets/bernie_tweets.pkl', 'w') as picklefile:
#     pickle.dump(bernie_tweets, picklefile)

# with open('tweets/rubio_tweets.pkl', 'w') as picklefile:
#     pickle.dump(rubio_tweets, picklefile)

# with open('tweets/cruz_tweets.pkl', 'w') as picklefile:
#     pickle.dump(cruz_tweets, picklefile)
    
# with open('tweets/kanye_tweets.pkl', 'w') as picklefile:
#     pickle.dump(kanye_tweets, picklefile)

#### 02/29 update

In [2]:
# define last_id, manual here; should create function to get this later...
last_id = {'clinton': 702556529518628864,
 'cruz': 702556529518628864,
 'rubio': 702556529518628864,
 'sanders': 702556529518628864,
 'trump': 702556529518628864}

In [69]:
# get recent tweets and insert into mongodb 
for cand in candidates:
    results = get_tweets(candidates[cand], since_id = last_id[cand])
    into_mongo(coll_dict[cand], results)

1
2
1
2
1
2
1
2
1
2


### Get retweets - assess sentiment in response to tweets and who is retweeting

In [249]:
"""probably not a great idea since all retweets are likely to be positive and only returns latest 
100 retweets of the tweet. better to just get followers instead?"""

'probably not a great idea since all retweets are likely to be positive and only returns latest \n100 retweets of the tweet. better to just get followers instead?'

In [203]:
with open('tweets/donald_tweets.pkl', 'r') as picklefile:
    dt_data = pickle.load(picklefile)

In [214]:
def get_ids(cand_dict):
    ids_list = [cand_dict[i]['id'] for i in range(len(cand_dict))]
    return ids_list

In [217]:
dt_ids

[702561529447763968,
 702560558269202433,
 702542084776001537,
 702539414224936960,
 702526610646429696,
 702525836948336641,
 702519153446400000,
 702484363695370241,
 702378896621236224,
 702367612232142848,
 702312059690622976,
 702311980732813314,
 702311041762332672,
 702310109100126208,
 702310025235042304,
 702308297886400513,
 702270277560852480,
 702268900562448384,
 702194423048699905,
 702180291448680448,
 702178985430503424,
 702172874258264064,
 702172591813857280,
 702158475091021828,
 702157754283130884,
 702155553485619200,
 702155147695284224,
 702149896325890048,
 702147760926011393,
 702141081027104769,
 702140146343280640,
 701999802033111040,
 701977550315098113,
 701967889457229828,
 701957484328583168,
 701956476571512832,
 701954385404760064,
 701945579333513216,
 701944029315923968,
 701943239054532608,
 701941907052634112,
 701941803327426560,
 701940652339122176,
 701929835132641282,
 701924334621716480,
 701876939095543808,
 701875717504536576,
 701872910764

In [218]:
def get_retweets(ids_list):
    results = []
    for i in ids_list:
        search_params = {'id': i, 'count': 100}
        json = requests.get('https://api.twitter.com/1.1/statuses/retweets/%i.json' %i,
                    params = search_params,
                    auth = oauth).json()
        results.append(json)
    return results

In [243]:
dt_retweets = get_retweets([702560558269202433])

In [245]:
dt_retweets[87]['text']

u'RT @realDonaldTrump: In all of television- the only one who said anything bad about last nights landslide victory-- was dopey @KarlRove. He\u2026'

In [244]:
len(dt_retweets)

94

In [224]:
dt_retweets_df = pd.DataFrame(dt_retweets)

In [237]:
dt_retweets_df.shape

(87, 26)

In [234]:
dt_retweets[0]['user']

{u'contributors_enabled': False,
 u'created_at': u'Sun Mar 01 19:22:27 +0000 2009',
 u'default_profile': False,
 u'default_profile_image': False,
 u'description': u'I ALREADY KNOW d Chicago politics Pres Obama MARRIED into & I wait 2 see if my #BossDaley or #TrumpCard political deal will be done & my @BrookebCNN #marriage',
 u'entities': {u'description': {u'urls': []},
  u'url': {u'urls': [{u'display_url': u'facebook.com',
     u'expanded_url': u'http://facebook.com',
     u'indices': [0, 22],
     u'url': u'http://t.co/3luzodugcf'}]}},
 u'favourites_count': 3272,
 u'follow_request_sent': False,
 u'followers_count': 1170,
 u'following': False,
 u'friends_count': 2192,
 u'geo_enabled': True,
 u'has_extended_profile': True,
 u'id': 22394806,
 u'id_str': u'22394806',
 u'is_translation_enabled': False,
 u'is_translator': False,
 u'lang': u'en',
 u'listed_count': 262,
 u'location': u'Chicago Illinois',
 u'name': u'Earl Worthington',
 u'notifications': False,
 u'profile_background_color': u'

In [247]:
for tweet in dt_retweets:
    print tweet['user']['description']

Let us form one body, one heart, and defend to the last warrior our country, our homes, our liberty, and the graves of our fathers.

Cynical optimist
I ALREADY KNOW d Chicago politics Pres Obama MARRIED into & I wait 2 see if my #BossDaley or #TrumpCard political deal will be done & my @BrookebCNN #marriage

Showcasing all the #Boss moments from #Trump2016



fighting for E3, Eternal Equality Everywhere freedom fighter, equalist, dancer, writer, atheist, darwinist, vegan, gay, intersexual, animal lover....
UNC Department of Public Safety Security Officer. IG- dallascarpenter

#Luxury Influencer, Coolhunter & Fashion Blogger #Instagram: PenelopeTrendy
Director for , TRUE VEGAS NIGHTS

I love my Lord and Savior, Jesus Christ, Conservative, Pro 2nd, profamily America the Beautiful Never perfect Always trying, Know I'm pleased w/my Trump hat❤
He will run the table & dominate Hillary. #BlueLivesMatter. Followed by @mitchellvii and @DRJamesCabot.  Post your #TrumpFive and I will follow you!


### get followers by candidate

In [ ]:
"""method 1 - get follower ids > user lookup by id
1) get follower ids: 5000 ids per request, 15 requests per 15 min | 75,000 followers per 15 min = 20 hours for 6 mil followers
2) user lookup by id: 100 users per request, 180 requests per 15 min = 13 hours per 1 mil

method 2 - get users/lookup: https://dev.twitter.com/rest/reference/get/users/lookup
200 users per request, 15 requests per 15 min = 3000 per 15 min = 83 hours for 1 million
"""

In [268]:
((1000000/3000)*15)/60

83

In [264]:
1000000/200*15*15/60

18750

### get mentions by location - state

In [279]:
gop = ['alabama', 'georgia', 'mass', 'oklahoma', 'texas', 'virginia', 'michigan']
dem = ['arkansas', 'georgia', 'mass', 'oklahoma', 'texas', 'vermont', 'virginia', 'michigan']

In [280]:
states = list(set(gop + dem))

In [281]:
states

['vermont',
 'georgia',
 'oklahoma',
 'virginia',
 'mass',
 'arkansas',
 'alabama',
 'michigan',
 'texas']

In [291]:
def get_mentions(term):
    search_params = {'q': term, 
                     'geocode': '33.2588817,-86.8295336,207km',
                     'count': 100}
    json = requests.get('https://api.twitter.com/1.1/search/tweets.json',
                params = search_params,
                auth = oauth).json()
    return json

In [292]:
blah = get_mentions('@realdonaldtrump')

In [305]:
for tweet in blah['statuses']:
    print tweet['text']

RT @Captain_Strongo: This is much better than the one I posted in last night’s #GOPDebate:

@marcorubio @tedcruz @realDonaldTrump https://t…
This is much better than the one I posted in last night’s #GOPDebate:

@marcorubio @tedcruz @realDonaldTrump https://t.co/PO1th4KBTq
1) So @realDonaldTrump knew Rubio was going to bring it at the debate so prior to it he secures @ChrisChristie endorsement.
Ummm... RT @CNN: BREAKING: @ChrisChristie endorses @realDonaldTrump https://t.co/sAaWx9yVF7  https://t.co/0IeCfm6KvZ
@realDonaldTrump Donald stant your ground .Cruse and Rubio have had time in the political arena to have made a change.
@LisaBlackman7 @INJO @marcorubio @GMA @realDonaldTrump You mean CPAC, which he is in fact attending?
@CNNPolitics @realDonaldTrump @teddyschleifer VOTE HILLARY CLINTON 2016. AMERICA IS READY .
@realDonaldTrump @classyexplorer @ahernandez85a Didn't they give Romney a participation trophy for coming in 2nd place in 2012?
@realDonaldTrump on your terms. You are kicki

In [300]:
blah['statuses'][0]['geo']

In [286]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Alabama")
print location.latitude, location.longitude

33.2588817 -86.8295336


In [283]:
print location

Flatiron Building, 175, 5th Avenue, Flatiron, Manhattan, New York County, NYC, New York, 10010, United States of America


In [269]:
import math

In [274]:
def get_radius(area):
    return math.sqrt(area / math.pi)

In [277]:
get_radius(135765)

207.88300002102707

**Get current rate limit usage.**

In [75]:
def get_remaining_calls():
    """Get remaining calls for statuses/user_timeline requests."""
    usage = requests.get('https://api.twitter.com/1.1/application/rate_limit_status.json?resources=statuses', 
                     auth = oauth).json()
    detail = usage['resources']['statuses']['/statuses/user_timeline']
    next_reset = datetime.datetime.fromtimestamp(detail['reset']).strftime('%Y-%m-%d %H:%M:%S')
    return detail['remaining'], next_reset

In [192]:
remaining_calls, next_reset = get_remaining_calls()

In [193]:
print remaining_calls
print next_reset

162
2016-02-24 15:07:09


#### Import Tweets into MongoDB

In [39]:
hmm = "trump_tweets"

In [50]:
client = MongoClient()
db = client.tweets
coll_dict = {'trump': db.trump, 
             'clinton' : db.clinton, 
             'sanders' : db.sanders, 
             'cruz' : db.cruz, 
             'rubio' : db.rubio}

def into_mongo(collection, results):
    coll = collection
    for tweet in results:
        coll.insert_one(tweet)

In [49]:
into_mongo(coll_dict['trump'], donald_0228)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [25]:
# Our collection: 
tweets = db.trump_tweets

In [26]:
for tweet in donald_0228:
    tweets.insert(tweet)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  from ipykernel import kernelapp as app


In [24]:
for tweet in donald_0228:
    data={}
    data['tweet']=tweet.text.encode('utf-8') 
    data['datetime']=tweet.created_at
    tweets.insert(data)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [51]:
tweet_list = []

In [64]:
def out_of_mongo(collection):
    tweet_list = []
    for tweet in collection.find():
        tweet_list.append(tweet)
    return tweet_list

In [55]:
tweet_list

[{u'_id': ObjectId('56d4731b4522e06e583672de'),
  u'contributors': None,
  u'coordinates': None,
  u'created_at': u'Mon Feb 29 16:16:27 +0000 2016',
  u'entities': {u'hashtags': [{u'indices': [0, 10], u'text': u'Trump2016'},
    {u'indices': [11, 26], u'text': u'TrumpInstagram'}],
   u'media': [{u'display_url': u'pic.twitter.com/BpKZcISeKi',
     u'expanded_url': u'http://twitter.com/realDonaldTrump/status/704339457974132736/photo/1',
     u'id': 704339436167831553L,
     u'id_str': u'704339436167831553',
     u'indices': [52, 75],
     u'media_url': u'http://pbs.twimg.com/media/CcZQ_EYVIAESGsT.jpg',
     u'media_url_https': u'https://pbs.twimg.com/media/CcZQ_EYVIAESGsT.jpg',
     u'sizes': {u'large': {u'h': 1024, u'resize': u'fit', u'w': 1024},
      u'medium': {u'h': 600, u'resize': u'fit', u'w': 600},
      u'small': {u'h': 340, u'resize': u'fit', u'w': 340},
      u'thumb': {u'h': 150, u'resize': u'crop', u'w': 150}},
     u'type': u'photo',
     u'url': u'https://t.co/BpKZcISeKi'}

In [35]:
ugh_df = pd.DataFrame(tweet_list)

In [36]:
ugh_df.head()

,_id,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,56d4731b4522e06e583672de,None,None,Mon Feb 29 16:16:27 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",{u'media': [{u'expanded_url': u'http://twitter...,973,False,None,704339457974132736,...,NaN,NaN,NaN,454,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",#Trump2016 #TrumpInstagram: https://t.co/tzHtn...,False,"{u'follow_request_sent': False, u'has_extended..."
1,56d4731b4522e06e583672df,None,None,Mon Feb 29 15:49:03 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",{u'media': [{u'expanded_url': u'http://twitter...,3004,False,None,704332561296855040,...,NaN,NaN,NaN,1114,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",I am self funding my campaign and only work fo...,False,"{u'follow_request_sent': False, u'has_extended..."
2,56d4731b4522e06e583672e0,None,None,Mon Feb 29 15:17:37 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,3750,False,None,704324651384762368,...,NaN,NaN,NaN,1424,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","""Hillary could lose to Trump in Democratic New...",False,"{u'follow_request_sent': False, u'has_extended..."
3,56d4731b4522e06e583672e1,None,None,Mon Feb 29 14:28:43 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,3459,False,None,704312346706124800,...,NaN,NaN,NaN,1178,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","Join me in Columbus, Ohio tomorrow!\n#MakeAmer...",False,"{u'follow_request_sent': False, u'has_extended..."
4,56d4731b4522e06e583672e2,None,None,Mon Feb 29 12:43:13 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",{u'media': [{u'expanded_url': u'http://twitter...,6101,False,None,704285794643668992,...,NaN,NaN,NaN,2303,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Thank you America! Get out &amp; VOTE tomorrow...,False,"{u'follow_request_sent': False, u'has_extended..."
